In [56]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SQLContext

import spacy
import string

import re
import gensim
from gensim import corpora

import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

In [4]:
book_review_df = spark.read.json('data/reviews_Books.json')
comics_df = spark.read.json('data/comic_reviews_wtitle.json')

In [12]:
comics_asins = [asin[0] for asin in comics_df.select('asin').dropDuplicates().collect()]

In [15]:
book_review_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [18]:
# Filtering review text for comic books used from all book reviews
comic_review_text  = book_review_df.select(['asin',
                                            'reviewText',
                                            'summary'
                                           ]).filter(col("asin").isin(comics_asins)).collect()

In [23]:
# Converting individual reviews to dictionary of ASIN with all review text.
review_text = {}
for review in comic_review_text:
    if review[0] not in review_text.keys():
        review_text[review[0]] = review[1] + review[2]
    else:
        review_text[review[0]] += review[1] + review[2]

In [4]:
# Temporarily saving to CSV
reviews_df = pd.read_csv('data/review_text.csv', index_col=0)

In [5]:
reviews_df.head()

,asin,text
0,0316107255,PENGUIN DREAMS AND STRANGER THINGS is an engag...
1,0345507460,"Well, I'm sorry there's people who didn't enjo..."
2,0345529375,beautiful story with bryan's wonderful and awk...
3,0375424148,"The drawings are somtimes almost psychedelic, ..."
4,0375424334,"If you like Chris Ware this is a must, the usu..."


In [57]:
# Functions to clean and tokenize text
def replace_punct_and_numbers(text):
    """Remove punctuation from document"""
    punct = [punc for punc in string.punctuation]
    num = list(range(10))
    clean_text = "".join([letter for letter in text if (letter not in punct) and (letter not in num)])
    return clean_text

def remove_ner(text):
    """Remove NER words from text"""
    doc_nlp = nlp(text)
    ner_words = [word.text for word in doc_nlp.ents]
    ner_regex = re.compile("|".join(ner_words))
    doc_ner_clean = ner_regex.sub('', text)
    return doc_ner_clean

def clean(doc):
    """Process text and return tokenized"""
    punct_free = replace_punct_and_numbers(doc)
    doc_ready = nlp(remove_ner(punct_free))
    tokens = [token.lemma_.lower() for token in doc_ready 
                  if (not token.is_stop) and (token.text.strip() != "")]
    return tokens

In [31]:
texts = [clean(doc) for doc in reviews_df['text'].tolist()]

In [33]:
dictionary = corpora.Dictionary(texts)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in texts]

In [33]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes=50)

In [44]:
print(ldamodel.print_topics(num_topics=5, num_words=25))

[(0, '0.001*"odst" + 0.000*"calender" + 0.000*"greenyellow" + 0.000*"index" + 0.000*"handbook" + 0.000*"ilver" + 0.000*"kindergartner" + 0.000*"adventures34" + 0.000*"ballet" + 0.000*"dorrie" + 0.000*"evans" + 0.000*"omega" + 0.000*"helljumper" + 0.000*"stein" + 0.000*"cole" + 0.000*"nova" + 0.000*"carlet" + 0.000*"hadow" + 0.000*"2099" + 0.000*"odsts" + 0.000*"madame" + 0.000*"vas" + 0.000*"baron" + 0.000*"adorableness" + 0.000*"thembirthday"'), (1, '0.024*"story" + 0.024*"book" + 0.014*"read" + 0.014*"not" + 0.012*"s" + 0.010*"like" + 0.010*"comic" + 0.009*"good" + 0.009*"great" + 0.008*"character" + 0.008*"art" + 0.007*"series" + 0.007*"comic_strip" + 0.006*"love" + 0.006*"fan" + 0.006*"time" + 0.005*"novel" + 0.005*"volume" + 0.005*"work" + 0.005*"issue" + 0.005*"graphic" + 0.005*"look" + 0.005*"page" + 0.004*"buy" + 0.004*"come"'), (2, '0.020*"story" + 0.017*"s" + 0.014*"not" + 0.012*"character" + 0.011*"book" + 0.011*"good" + 0.010*"like" + 0.008*"issue" + 0.008*"read" + 0.008*"s

In [45]:
ldamodel.save('data/lda.model')

In [52]:
ldamodel.get_document_topics(doc_term_matrix[1])

[(1, 0.8621955), (2, 0.13542761)]

In [49]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel_3topics = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes=50)

In [54]:
print(ldamodel_3topics.print_topics(num_topics=3, num_words=25))

[(0, '0.022*"story" + 0.018*"book" + 0.014*"not" + 0.014*"s" + 0.011*"read" + 0.010*"good" + 0.010*"like" + 0.010*"character" + 0.008*"great" + 0.007*"series" + 0.007*"issue" + 0.007*"comic" + 0.006*"art" + 0.006*"volume" + 0.006*"time" + 0.005*"new" + 0.004*"love" + 0.004*"comic_strip" + 0.004*"fan" + 0.004*"work" + 0.004*"come" + 0.004*"get" + 0.004*"know" + 0.004*"look" + 0.004*"think"'), (1, '0.010*"vampire" + 0.002*"dale" + 0.002*"chef" + 0.002*"food" + 0.002*"album" + 0.001*"sushi" + 0.001*"rman" + 0.001*"di" + 0.001*"rgirl" + 0.001*"mandias" + 0.001*"foodie" + 0.001*"che" + 0.001*"87" + 0.000*"restaurant" + 0.000*"culinary" + 0.000*"il" + 0.000*"vamp" + 0.000*"odst" + 0.000*"e" + 0.000*"una" + 0.000*"211" + 0.000*"jeanclaude" + 0.000*"un" + 0.000*"outlaw" + 0.000*"del"'), (2, '0.001*"immigrant" + 0.001*"de" + 0.001*"que" + 0.001*"la" + 0.001*"nd" + 0.001*"en" + 0.001*"los" + 0.000*"el" + 0.000*"threeboote" + 0.000*"es" + 0.000*"calendar" + 0.000*"tht" + 0.000*"una" + 0.000*"immi

In [59]:
ldamodel_3topics.get_document_topics(doc_term_matrix[1])

[(0, 0.9998981)]

In [61]:
lsi_model = gensim.models.LsiModel(doc_term_matrix, num_topics=3, id2word=dictionary)

In [62]:
lsi_model.print_topics(num_topics=3, num_words=10)

[(0,
  '0.382*"story" + 0.375*"book" + 0.287*"not" + 0.287*"read" + 0.248*"s" + 0.191*"like" + 0.178*"character" + 0.178*"good" + 0.155*"comic" + 0.148*"great"'),
 (1,
  '0.395*"story" + 0.322*"s" + -0.294*"book" + -0.268*"read" + -0.245*"tv" + -0.237*"series" + -0.231*"zombie" + -0.216*"compendium" + -0.169*"love" + -0.154*"comic"'),
 (2,
  '0.389*"novel" + 0.319*"book" + 0.311*"graphic" + -0.308*"issue" + -0.268*"volume" + -0.239*"series" + 0.235*"story" + -0.163*"new" + -0.131*"s" + -0.124*"great"')]